In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

# ---------------- Synthetic dataset ----------------
np.random.seed(42)
N = 10000

route_planning = pd.DataFrame({
    'origin_x': np.random.uniform(0, 100, N),
    'origin_y': np.random.uniform(0, 100, N),
    'dest_x': np.random.uniform(0, 100, N),
    'dest_y': np.random.uniform(0, 100, N),
    'traffic_density': np.random.uniform(0, 1, N),
    'road_type': np.random.choice([1, 2, 3], N),
})
route_planning['distance'] = np.sqrt((route_planning['dest_x'] - route_planning['origin_x'])**2 +
                                     (route_planning['dest_y'] - route_planning['origin_y'])**2)
speed_base = {1:50, 2:40, 3:30}
route_planning['speed'] = route_planning['road_type'].map(speed_base) * np.random.uniform(0.8,1.2,N)
route_planning['travel_time'] = (route_planning['distance']/route_planning['speed'])*60*\
                                (1+route_planning['traffic_density']*np.random.uniform(0.1,0.5,N))

vehicle_assignment = pd.DataFrame({
    'vehicle_capacity': np.random.randint(50,200,N),
    'battery_level': np.random.uniform(0.3,1.0,N),
    'delivery_size': np.random.randint(5,50,N),
    'vehicle_type': np.random.choice([1,2],N),
    'speed_factor': np.random.uniform(0.9,1.1,N),
})
vehicle_assignment['assigned_speed'] = route_planning['speed']*vehicle_assignment['speed_factor']
vehicle_assignment['load_utilization'] = vehicle_assignment['delivery_size']/vehicle_assignment['vehicle_capacity']

time_scheduling = pd.DataFrame({
    'requested_time': np.random.randint(8,20,N),
    'delivery_priority': np.random.randint(1,5,N),
    'customer_patience': np.random.uniform(0,1,N),
})
time_scheduling['delay_probability'] = np.clip(
    (route_planning['travel_time']/60)*(1+vehicle_assignment['load_utilization']*0.5)*np.random.uniform(0.8,1.2,N),
    0,1
)

dynamic_rerouting = pd.DataFrame({
    'current_x': np.random.uniform(0,100,N),
    'current_y': np.random.uniform(0,100,N),
    'traffic_updates': np.random.uniform(0,1,N),
    'new_delivery_requests': np.random.randint(0,3,N),
    'vehicle_status': np.random.choice([0,1],N),
    'weather': np.random.choice([0,1],N),
})
dynamic_rerouting['congestion_score'] = dynamic_rerouting['traffic_updates'] + \
                                       dynamic_rerouting['new_delivery_requests']*0.5 + \
                                       dynamic_rerouting['weather']*0.5 + \
                                       (route_planning['travel_time']/route_planning['travel_time'].max())*0.5

# ---------------- Combine and normalize ----------------
datasets = [route_planning, vehicle_assignment, time_scheduling, dynamic_rerouting]
dataset_dims = [df.shape[1] for df in datasets]
max_dim = max(dataset_dims)

padded_data = []
for df in datasets:
    arr = df.values
    if arr.shape[1] < max_dim:
        arr = np.hstack([arr, np.zeros((arr.shape[0], max_dim - arr.shape[1]))])
    padded_data.append(arr)

DATA_MATRIX = np.hstack(padded_data)
DATA_MATRIX = (DATA_MATRIX - DATA_MATRIX.min(axis=0)) / (np.ptp(DATA_MATRIX, axis=0)+1e-8)

def generate_targets(DATA_MATRIX, candidate_dims, D_graph):
    targets = []
    for node_idx in range(D_graph):
        row = DATA_MATRIX[node_idx % len(DATA_MATRIX)]
        node_targets = {}
        for dim in candidate_dims:
            if len(row) >= dim:
                sampled = row[:dim]
            else:
                sampled = np.pad(row, (0, dim - len(row)), constant_values=0.5)
            node_targets[dim] = sampled
        targets.append(node_targets)
    return targets
candidate_dims = [6,6,6,6,6,6]#,6,6,6,6,6,6,6,6,6,6,6,6,]
D_graph = 4
synthetic_targets = generate_targets(DATA_MATRIX, candidate_dims, D_graph)

# ---------------- Targets ----------------
#candidate_dims = [6,6,6,6,6,6,6]#,6,6,6,6,6,6,6,6,6,6,6,6,]
#D_graph = 4
inner_archive_size = 120
inner_offspring = 80
outer_archive_size = 80
outer_offspring = 80
inner_iters_per_outer = 15
outer_generations = 500
outer_cost_limit = 1350
inner_learning = 0.9
seed = 42
np.random.seed(seed)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.cm import get_cmap
import torch
import torch.nn as nn
import torch.nn.functional as F
import seaborn as sns


# ---------------- CONFIG ----------------
candidate_dims = [6] * 3
D_graph = 4

inner_archive_size = 80
inner_offspring = 40
outer_archive_size = 40
outer_offspring = 40
inner_iters_per_outer = 50
outer_generations = 101
outer_cost_limit = 1000
inner_learning = 0.1
gamma_interlayer = -0.5
seed = 42
np.random.seed(seed)
import torch
import torch.nn as nn
import torch.nn.functional as F

# ---------------------- Neural Nets for Metric Estimation ----------------------
class ParamNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.out = nn.Linear(hidden_dim // 2, 3)
        self.reset_parameters()

    def reset_parameters(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=0.5)
                nn.init.constant_(m.bias, 0.0)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        raw = self.out(h)
        w0 = F.softplus(raw[:, 0])
        t0 = F.softplus(raw[:, 1])
        u0 = torch.sigmoid(raw[:, 2])
        w0 = 10.0 * (1.0 + 0.2 * (w0 - 1.0))
        t0 = 100.0 * (1.0 + 0.2 * (t0 - 1.0))
        u0 = 1.0 * u0
        return torch.stack([w0, t0, u0], dim=1)


class PatienceNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=32):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc_out = nn.Linear(hidden_dim // 2, 1)

        # Weight initialization
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=0.5)
                nn.init.constant_(m.bias, 0.0)

    def forward(self, x, adj):
        # Add self-loops
        adj = adj + torch.eye(adj.size(0), device=adj.device)
        # First GNN layer
        h = torch.matmul(adj, x)
        h = F.relu(self.fc1(h))
        # Second GNN layer
        h = torch.matmul(adj, h)
        h = F.relu(self.fc2(h))
        # Output per node
        raw = self.fc_out(h)
        patience = torch.sigmoid(raw).squeeze(-1)
        # Graph-level output: mean over nodes
        return patience.mean(dim=0, keepdim=True)

# ---------------------- MetricsEvaluator using Neural Estimators ----------------------
class MetricsEvaluator:
    def __init__(self, data_matrix, param_net: ParamNet, patience_net: PatienceNet, device='cpu'):
        self.data_matrix = data_matrix.astype(np.float32)
        self.num_nodes = data_matrix.shape[0]
        self.num_features = data_matrix.shape[1]
        self.device = device
        self.param_net = param_net.to(self.device)
        self.patience_net = patience_net.to(self.device)
        self.param_net.eval()
        self.patience_net.eval()

    def compute_node_metrics(self, node_idx, y=None, use_graph=False):
        """
        Compute metrics for a node, optionally using adjacency.
        node_idx: index of the node
        y: optional target vector
        use_graph: if True, pass adjacency to GNN
        """
        row = self.data_matrix[node_idx % self.num_nodes]
        row_tensor = torch.from_numpy(row).unsqueeze(0).to(self.device)

        with torch.no_grad():
            params = self.param_net(row_tensor).cpu().numpy().squeeze(0)

            if use_graph:
                # single node adjacency = 1x1 identity
                adj = torch.eye(1, device=self.device)
                patience_val = float(self.patience_net(row_tensor, adj).cpu().numpy().squeeze(0))
            else:
                # old behavior
                # for a single node, still 1x1 identity inside PatienceNet
                adj = torch.eye(1, device=self.device)
                patience_val = float(self.patience_net(row_tensor, adj).cpu().numpy().squeeze(0))

        W0_pred, T0_pred, U0_pred = params

        if y is None:
            y = np.array([0.5, 0.5, 0.5])
        else:
            y = np.array(y[:3]) if len(y) >= 3 else np.pad(y, (0, 3-len(y)), constant_values=0.5)

        # Compute signals
        k = self.num_features
        base = max(1, k//3)
        wait_signal = np.mean(row[0:base])
        throughput_signal = np.mean(row[base:2*base])
        util_signal = np.mean(row[2*base:k])

        wait = np.clip(W0_pred * (1 + 1.2*wait_signal + 0.8*y[0]), 0, 100)
        throughput = np.clip(T0_pred * (1 + 1.1*throughput_signal + 0.6*y[1] - 0.4*wait_signal), 0, 150)
        util = np.clip(U0_pred + 0.8*util_signal + 0.6*y[2], 0, 1)
        score = -wait + throughput + util + patience_val

        return {
            'wait': float(wait),
            'throughput': float(throughput),
            'util': float(util),
            'patience': patience_val,
            'score': float(score),
            'W0': W0_pred,
            'T0': T0_pred,
            'U0': U0_pred
        }

    def compute_graph_patience(self, nodes_idx=None):
        """
        Compute patience for multiple nodes at once using adjacency.
        nodes_idx: list of node indices
        """
        if nodes_idx is None:
            nodes_idx = list(range(self.num_nodes))

        x = torch.from_numpy(self.data_matrix[nodes_idx]).to(self.device)
        # Fully connected adjacency + self-loops
        adj = torch.ones(len(nodes_idx), len(nodes_idx), device=self.device)
        patience_val = self.patience_net(x, adj).cpu().numpy().squeeze()
        return patience_val


# ---------------- INTER-LAYER ----------------
class InterLayer:
    def __init__(self, D_graph, max_inner_dim, inter_dim=None, edge_threshold=0.02, gamma=1.0, seed=42):
        np.random.seed(seed)
        self.D_graph = D_graph
        self.max_input = 2*max_inner_dim
        self.edge_threshold = edge_threshold
        self.gamma = gamma
        self.inter_dim = inter_dim if inter_dim is not None else max_inner_dim
        self.weights = {(i,j): np.random.uniform(-0.6,0.6,(self.inter_dim,self.max_input))
                        for i in range(D_graph) for j in range(D_graph) if i!=j}
        self.bias = {(i,j): np.random.uniform(-0.3,0.3,self.inter_dim)
                     for i in range(D_graph) for j in range(D_graph) if i!=j}
    # ---------- INTER-LAYER ACTIVATIONS ----------
    def print_interactions(self, return_tensor=True, verbose=True):
        """
        Print or return the inter-layer activation tensor.
        Shape: (D_graph, D_graph, inter_dim)
        """
        D_graph = self.D_graph
        inter_dim = self.inter_layer.inter_dim
        inter_tensor = np.zeros((D_graph, D_graph, inter_dim))
        acts = self.inter_layer.build_activations(self.chosen_Gmat, self.nested_reps)
        if not acts:
            if verbose: print("No active edges above threshold.")
            return inter_tensor if return_tensor else None
        for (i, j), vec in acts.items():
            inter_tensor[i, j, :] = vec
            if verbose:
                act_str = ", ".join([f"{v:.3f}" for v in vec])
                print(f"Node {i} -> Node {j}: [{act_str}]")
        return inter_tensor if return_tensor else None

    def compute_edge_activation(self, i, j, nested_reps):
        concat = np.concatenate([nested_reps[i], nested_reps[j]])
        concat = np.pad(concat, (0, max(0,self.max_input-len(concat))))[:self.max_input]
        v = self.weights[(i,j)].dot(concat) + self.bias[(i,j)]
        return 1/(1+np.exp(-v))

    def build_activations(self, Gmat, nested_reps):
        acts = {}
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i==j: continue
                if abs(Gmat[i,j])>self.edge_threshold:
                    acts[(i,j)] = self.compute_edge_activation(i,j,nested_reps)
        return acts

    @staticmethod
    def pairwise_squared_corr(acts):
        if len(acts)<2: return 0.0
        A = np.stack(list(acts.values()))
        A_centered = A - A.mean(axis=1,keepdims=True)
        stds = np.sqrt(np.sum(A_centered**2,axis=1)/(A.shape[1]-1) + 1e-12)
        cov = A_centered @ A_centered.T / (A.shape[1]-1)
        corr = cov / (np.outer(stds,stds)+1e-12)
        np.fill_diagonal(corr,0)
        return float((corr**2).sum())

    def mi_for_graph(self, Gmat, nested_reps):
        acts = self.build_activations(Gmat,nested_reps)
        if not acts: return 0.0
        return self.gamma * self.pairwise_squared_corr(acts)

# ---------------- GDFCM CLASS ----------------
class GDFCM:
    def __init__(self, candidate_dims, D_graph, inner_archive_size, inner_offspring,
                 outer_archive_size, outer_offspring, synthetic_targets, inner_learning,
                 gamma_interlayer=1.0, causal_flag=True):
        self.candidate_dims = candidate_dims
        self.D_graph = D_graph
        self.inner_archive_size = inner_archive_size
        self.inner_offspring = inner_offspring
        self.outer_archive_size = outer_archive_size
        self.outer_offspring = outer_offspring
        self.synthetic_targets = synthetic_targets
        self.inner_learning = inner_learning
        self.causal_flag = causal_flag

        self.nested_reps = [np.zeros(max(candidate_dims)) for _ in range(D_graph)]
        self.best_dim_per_node = [candidate_dims[0] for _ in range(D_graph)]
        self.inter_layer = InterLayer(D_graph, max_inner_dim=max(candidate_dims), gamma=gamma_interlayer)
        self.chosen_Gmat = np.random.uniform(-0.5,0.5,(D_graph,D_graph))
        np.fill_diagonal(self.chosen_Gmat,0)
        self.l2_before, self.l2_after = [], []

        # Store capped node metrics for plotting and fuzzy tensor
        self.capped_node_metrics = 250

    # ---------- INNER LOOP ----------
    def run_inner(self, node_idx, target, D_fcm, steps=100, lr_x=0.001, lr_W=0.001):
        x = target.copy()
        W = np.random.uniform(-0.6,0.6,(D_fcm,D_fcm))
        np.fill_diagonal(W,0)
        self.l2_before.append(np.linalg.norm(self.nested_reps[node_idx]-target))

        for _ in range(steps):
            z = W.dot(x)
            Theta_grad_z = 2*z - target
            Theta_grad_x = Theta_grad_z @ W + 0.5*(x+1)**2
            Theta_grad_W = np.outer(Theta_grad_z,x)

            x -= lr_x * np.clip(Theta_grad_x,-0.05,0.05)
            x = np.clip(x,0,1)
            W -= lr_W * np.clip(Theta_grad_W,-0.01,0.01)
            np.fill_diagonal(W,0)
            W = np.clip(W,-1,1)

        self.nested_reps[node_idx] = x
        self.l2_after.append(np.linalg.norm(x-target))
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat,self.nested_reps)
        return x, W, x, mi_score

    # ---------- OUTER LOOP ----------
    def run_outer(self, outer_cost_limit=100):
        """
        Compute node metrics, integrate Fuzzy Metric Tensor (FMT) into the score,
        and apply capping if total exceeds outer_cost_limit.
        """
        metrics_evaluator = MetricsEvaluator(DATA_MATRIX, param_net, patience_net)
        node_metrics_list = []
        raw_scores = []

        # --- Compute node metrics per node ---
        for i, y in enumerate(self.nested_reps):
            metrics = metrics_evaluator.compute_node_metrics(i, y=y)
            node_metrics_list.append(metrics)
            raw_scores.append(metrics['score'])

        raw_scores = np.array(raw_scores)
        total_raw = raw_scores.sum()

        # --- Compute Fuzzy Metric Tensor ---
        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)
        D = self.D_graph
        # Only consider off-diagonal entries to capture inter-node interactions
        off_diag_mask = np.ones((D,D), dtype=bool)
        np.fill_diagonal(off_diag_mask, 0)
        fuzzy_score_offdiag = fuzzy_tensor[off_diag_mask].sum()

        # --- Combine raw score with fuzzy contribution ---
        combined_score = total_raw + self.inter_layer.gamma * fuzzy_score_offdiag

        # --- Apply cap if needed ---
        if combined_score > outer_cost_limit:
            scale_factor = outer_cost_limit / combined_score
            for metrics in node_metrics_list:
                for key in ['wait', 'throughput', 'util', 'patience', 'score']:
                    metrics[key] *= scale_factor
            combined_score = outer_cost_limit

        # --- Store capped metrics for plotting / fuzzy tensor ---
        self.capped_node_metrics = node_metrics_list

        return node_metrics_list, combined_score





    # ---------- FULL RUN ----------
    def run(self, outer_generations=101, outer_cost_limit=1000):
        """
        Full optimization run over outer generations.
        Uses Fuzzy Metric Tensor to adjust outer score dynamically.
        """
        final_metrics = None

        for gen in range(outer_generations):
            mi_scores = []

            # --- Inner loop per node ---
            for node_idx in range(self.D_graph):
                dim = self.best_dim_per_node[node_idx]
                target = self.synthetic_targets[node_idx][dim]
                _, _, _, mi_score = self.run_inner(node_idx, target, dim)
                mi_scores.append(mi_score)

            # --- Outer loop: metrics + fuzzy contribution ---
            metrics_list, combined_score = self.run_outer(outer_cost_limit=outer_cost_limit)

            # --- Print metrics ---
            print(f"\n--- Generation {gen} Metrics ---")
            for i, m in enumerate(metrics_list):
                print(
                    f"Node {i} | "
                    + " | ".join([f"{k}: {v:.2f}" for k, v in m.items()])
                )
            print(f"Outer Score (capped & FMT-adjusted): {combined_score:.3f}")

            final_metrics = metrics_list

        return final_metrics


    # ---------- PLOTTING FUNCTIONS USE CAPPED METRICS ----------
    def _get_metrics_for_plotting(self, i):
        # Return capped metrics if available
        if self.capped_node_metrics is not None:
            return self.capped_node_metrics[i]
        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)
        return metrics_evaluator.compute_node_metrics(i, y=self.nested_reps[i])

    def plot_metric_dashboard_colored(self):
        metrics_keys = ['wait', 'throughput', 'util', 'patience']
        D_graph = self.D_graph
        metric_matrix = np.zeros((D_graph, len(metrics_keys)))
        for i in range(D_graph):
            metrics = self._get_metrics_for_plotting(i)
            metric_matrix[i, :] = [metrics[k] for k in metrics_keys]

        metric_norm = (metric_matrix - metric_matrix.min(axis=0)) / (np.ptp(metric_matrix, axis=0) + 1e-12)
        cmap = get_cmap("viridis")

        fig, axes = plt.subplots(3, 1, figsize=(12, 8), gridspec_kw={'height_ratios':[1,1,1]})

        bottoms = np.zeros(D_graph)
        for idx, key in enumerate(metrics_keys):
            colors = [cmap(val) for val in metric_norm[:, idx]]
            axes[0].bar(range(D_graph), metric_matrix[:, idx], bottom=bottoms, color=colors,
                        edgecolor='black', label=key)
            bottoms += metric_matrix[:, idx]
        axes[0].set_xticks(range(D_graph))
        axes[0].set_xticklabels([f"Node {i+1}" for i in range(D_graph)])
        axes[0].set_ylabel("Metric Value (Stacked)")
        axes[0].set_title("Stacked Node Metrics (Color by Intensity)")
        axes[0].legend(loc='upper right')

        for i, rep in enumerate(self.nested_reps):
            axes[1].plot(range(len(rep)), rep + i*1.1, '-o', label=f"Node {i+1}")
        axes[1].set_ylabel("Nested Reps (shifted)")
        axes[1].set_title("Nested Representations per Node")
        axes[1].legend(loc='upper right')

        sns.heatmap(metric_matrix, annot=True, fmt=".2f", cmap="viridis",
                    yticklabels=[f"Node {i+1}" for i in range(D_graph)],
                    xticklabels=metrics_keys, ax=axes[2])
        axes[2].set_title("Node Metrics Heatmap")
        axes[2].set_xlabel("Metrics")
        axes[2].set_ylabel("Nodes")

        plt.tight_layout()
        plt.show()

    def plot_activations_and_metrics(self, normalize_activations=True):
        metrics_keys = ['wait', 'throughput', 'util', 'patience']
        D_graph = self.D_graph
        metric_matrix = np.zeros((D_graph, len(metrics_keys)))
        for i in range(D_graph):
            metrics = self._get_metrics_for_plotting(i)
            metric_matrix[i, :] = [metrics[k] for k in metrics_keys]

        plt.figure(figsize=(12, 6))
        for i, rep in enumerate(self.nested_reps):
            rep_vals = np.array(rep)
            rep_vals_norm = (rep_vals - rep_vals.min()) / (np.ptp(rep_vals) + 1e-12) if normalize_activations else rep_vals
            colors = plt.cm.plasma(rep_vals_norm)

            plt.subplot(2, D_graph, i+1)
            plt.bar(range(len(rep_vals)), rep_vals, color=colors, edgecolor='black')
            plt.ylim(0, 1 if normalize_activations else rep_vals.max()*1.1)
            plt.title(f"Node {i+1} Activations")
            plt.xlabel("Dimension")
            plt.ylabel("Value")

            plt.subplot(2, D_graph, i+1+D_graph)
            plt.bar(range(len(metrics_keys)), metric_matrix[i, :],
                    color=plt.cm.viridis(metric_matrix[i,:]/(metric_matrix[i,:].max()+1e-12)), edgecolor='black')
            plt.ylim(0, metric_matrix.max()*1.1)
            plt.xticks(range(len(metrics_keys)), metrics_keys, rotation=45)
            plt.title(f"Node {i+1} Metrics")
            plt.ylabel("Value")

        plt.tight_layout()
        plt.show()

    def plot_activations_and_metrics_n(self, normalize_activations=True, normalize_metrics=True):
        """
        Plot per-node bar plots for:
        1. Nested activations (colored bars)
        2. Metrics ('wait', 'throughput', 'util', 'patience') normalized across all nodes
        """
        metrics_evaluator = MetricsEvaluator(DATA_MATRIX, param_net, patience_net)
        metrics_keys = ['wait', 'throughput', 'util', 'patience']
        D_graph = self.D_graph

        # Collect metrics for all nodes
        metric_matrix = np.zeros((D_graph, len(metrics_keys)))
        for i, rep in enumerate(self.nested_reps):
            metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
            metric_matrix[i, :] = [metrics[k] for k in metrics_keys]

        # Normalize metrics across all nodes & metrics
        if normalize_metrics:
            metric_matrix_norm = (metric_matrix - metric_matrix.min()) / (np.ptp(metric_matrix) + 1e-12)
        else:
            metric_matrix_norm = metric_matrix

        plt.figure(figsize=(12, 6))

        for i, rep in enumerate(self.nested_reps):
            # --- Nested activations bar ---
            rep_vals = np.array(rep)
            if normalize_activations:
                rep_vals_norm = (rep_vals - rep_vals.min()) / (np.ptp(rep_vals) + 1e-12)
            else:
                rep_vals_norm = rep_vals
            colors = plt.cm.plasma(rep_vals_norm)

            plt.subplot(2, D_graph, i+1)
            plt.bar(range(len(rep_vals)), rep_vals, color=colors, edgecolor='black')
            plt.ylim(0, 1 if normalize_activations else rep_vals.max()*1.1)
            plt.title(f"Node {i+1} Activations")
            plt.xlabel("Dimension")
            plt.ylabel("Value")

            # --- Metrics bar (normalized across all nodes) ---
            plt.subplot(2, D_graph, i+1+D_graph)
            plt.bar(range(len(metrics_keys)), metric_matrix[i, :],
                    color=plt.cm.viridis(metric_matrix_norm[i, :]), edgecolor='black')
            plt.ylim(0, 1 if normalize_metrics else metric_matrix.max()*1.1)
            plt.xticks(range(len(metrics_keys)), metrics_keys, rotation=45)
            plt.title(f"Node {i+1} Metrics")
            plt.ylabel("Value")

        plt.tight_layout()
        plt.show()
    def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
        """
        Compute a fuzzy metric tensor for all node pairs (D_graph x D_graph x num_metrics),
        where each slice [i,j,:] contains metrics of node j (or combined metrics of i->j),
        optionally normalized across all nodes and metrics.
        """
        metrics_evaluator = MetricsEvaluator(DATA_MATRIX, param_net, patience_net)
        metrics_keys = ['wait', 'throughput', 'util', 'patience']
        D_graph = self.D_graph
        num_metrics = len(metrics_keys)

        tensor = np.zeros((D_graph, D_graph, num_metrics))

        # Compute metric vector for each node
        node_metrics = []
        for i, rep in enumerate(self.nested_reps):
            metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
            node_metrics.append(np.array([metrics[k] for k in metrics_keys]))

        node_metrics = np.array(node_metrics)  # shape: (D_graph, num_metrics)

        # Fill tensor: optionally weighted by edge strength in chosen_Gmat
        for i in range(D_graph):
            for j in range(D_graph):
                if i == j:
                    tensor[i, j, :] = node_metrics[j]  # self-metrics
                else:
                    weight = np.clip(abs(self.chosen_Gmat[i,j]), 0, 1)  # optional fuzzy weight
                    tensor[i, j, :] = weight * node_metrics[j]

        # Normalize tensor across all values (0-1)
        if normalize:
            tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

        if verbose:
            print("Fuzzy Metric Tensor shape:", tensor.shape)
            for i in range(D_graph):
                for j in range(D_graph):
                    print(f"Node {i} -> Node {j} metrics:", tensor[i,j,:])

        return tensor
    def compute_fuzzy_metric_tensor_bounds(self, verbose=False):
        """
        Compute a fuzzy metric tensor with lower and upper bounds per node pair.
        Uses capped metrics if available.
        Returns tensor of shape (D_graph, D_graph, num_metrics, 2)
        where [:,:,:,0] = lower, [:,:,:,1] = upper.
        """
        metrics_keys = ['wait', 'throughput', 'util', 'patience']
        D_graph = self.D_graph
        num_metrics = len(metrics_keys)

        tensor = np.zeros((D_graph, D_graph, num_metrics, 2))  # last dim: [lower, upper]

        # Use capped metrics if they exist
        if hasattr(self, 'capped_node_metrics') and self.capped_node_metrics is not None:
            node_metrics = np.array([[m[k] for k in metrics_keys] for m in self.capped_node_metrics])
        else:
            # fallback to raw metrics
            metrics_evaluator = MetricsEvaluator(DATA_MATRIX, param_net, patience_net)
            node_metrics = np.array([
                [metrics_evaluator.compute_node_metrics(i, y=self.nested_reps[i])[k] for k in metrics_keys]
                for i in range(D_graph)
            ])

        # Fuzzy bounds: for example ±10% around the metric value
        fuzz_factor = 0.1
        for i in range(D_graph):
            for j in range(D_graph):
                metrics_j = node_metrics[j]
                lower = np.clip(metrics_j * (1 - fuzz_factor), 0, None)
                upper = metrics_j * (1 + fuzz_factor)
                tensor[i, j, :, 0] = lower
                tensor[i, j, :, 1] = upper

        if verbose:
            print("Fuzzy Metric Tensor shape (with bounds):", tensor.shape)

        return tensor

    def plot_fuzzy_metric_tensor(self, fuzzy_tensor, metrics_keys=['wait','throughput','util','patience']):
        """
        Plot a visualization of the Fuzzy Metric Tensor (FMT) with lower/upper bounds.

        fuzzy_tensor: np.ndarray of shape (D_graph, D_graph, num_metrics, 2)
                    last dimension = [lower, upper]
        """
        D_graph = self.D_graph
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4*num_metrics, 4))

        for k in range(num_metrics):
            lower = fuzzy_tensor[:,:,k,0]
            upper = fuzzy_tensor[:,:,k,1]

            # Create a “fuzzy range” heatmap: color = mean, alpha = range/mean
            mean_vals = (lower + upper)/2
            range_vals = upper - lower
            # Normalize range for transparency
            max_range = range_vals.max() if range_vals.max()>0 else 1.0
            alphas = 0.2 + 0.8 * range_vals / max_range  # alpha 0.2–1.0

            # Plot mean as heatmap
            im = axes[k].imshow(mean_vals, cmap='viridis', vmin=0, vmax=mean_vals.max())

            # Overlay range as transparency mask
            for i in range(D_graph):
                for j in range(D_graph):
                    alpha_val = np.clip(1 - alphas[i,j], 0.0, 1.0)

                    rect = plt.Rectangle((j-0.5, i-0.5), 1, 1, color='white', alpha=alpha_val)

                    #rect = plt.Rectangle((j-0.5, i-0.5), 1, 1, color='white', alpha=1-alphas[i,j])
                    axes[k].add_patch(rect)
                    # Annotate with bounds
                    axes[k].text(j, i, f"{lower[i,j]:.1f}\n{upper[i,j]:.1f}", color='black',
                                ha='center', va='center', fontsize=9)

            axes[k].set_title(f"FMT - {metrics_keys[k]}")
            axes[k].set_xticks(range(D_graph))
            axes[k].set_xticklabels([f"Node {i+1}" for i in range(D_graph)])
            axes[k].set_yticks(range(D_graph))
            axes[k].set_yticklabels([f"Node {i+1}" for i in range(D_graph)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Mean Metric Value')
        plt.tight_layout()
        plt.show()

    def plot_fmt_score_contribution(self, metrics_keys=['wait','throughput','util','patience']):
        """
        Visualize Fuzzy Metric Tensor (FMT) and how it contributes to the outer score.
        Highlights inter-node impact on the score.
        """
        # Compute FMT
        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)
        D_graph = self.D_graph
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4*num_metrics, 4))

        for k in range(num_metrics):
            # Slice for this metric
            metric_tensor = fuzzy_tensor[:,:,k]

            # Mask diagonal to emphasize inter-node contribution
            masked_tensor = metric_tensor.copy()
            np.fill_diagonal(masked_tensor, 0)

            # Total contribution per node (row = node giving contribution)
            node_contrib = masked_tensor.sum(axis=1)

            im = axes[k].imshow(metric_tensor, cmap='viridis', vmin=0, vmax=1)
            for i in range(D_graph):
                for j in range(D_graph):
                    # Annotate with value and contribution
                    val = metric_tensor[i,j]
                    if i != j:
                        axes[k].text(j, i, f"{val:.2f}", ha='center', va='center', color='white', fontsize=9)
            axes[k].set_title(f"FMT - {metrics_keys[k]}")
            axes[k].set_xticks(range(D_graph))
            axes[k].set_xticklabels([f"Node {i+1}" for i in range(D_graph)])
            axes[k].set_yticks(range(D_graph))
            axes[k].set_yticklabels([f"Node {i+1}" for i in range(D_graph)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Normalized Metric Value')
        plt.tight_layout()
        plt.show()
    def plot_fmt_total_score(self):
        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)
        # Sum over metrics
        total_tensor = fuzzy_tensor.sum(axis=2)
        # Mask diagonal
        D_graph = self.D_graph
        np.fill_diagonal(total_tensor, 0)

        plt.figure(figsize=(5,5))
        sns.heatmap(total_tensor, annot=True, fmt=".2f",
                    xticklabels=[f"Node {i+1}" for i in range(D_graph)],
                    yticklabels=[f"Node {i+1}" for i in range(D_graph)],
                    cmap="viridis")
        plt.title("Total FMT Contribution to Score (Off-diagonal)")
        plt.xlabel("Node j")
        plt.ylabel("Node i")
        plt.show()
# ---------------- VISUALIZATIONS ----------------
# 1. Pointwise min/max elite vs. true targets
# ---------------- CONFIG ----------------
candidate_dims = [6] * 3
D_graph = 4
inner_archive_size = 80
inner_offspring = 40
outer_archive_size = 40
outer_offspring = 40
inner_learning = 0.1
gamma_interlayer = 0.3
outer_generations = 10  # for faster testing
outer_cost_limit = 1000
seed = 42
np.random.seed(seed)

# ---------------- INSTANTIATE OPTIMIZER ----------------
optimizer = GDFCM(
    candidate_dims=candidate_dims,
    D_graph=D_graph,
    inner_archive_size=inner_archive_size,
    inner_offspring=inner_offspring,
    outer_archive_size=outer_archive_size,
    outer_offspring=outer_offspring,
    synthetic_targets=synthetic_targets,
    inner_learning=inner_learning,
    gamma_interlayer=gamma_interlayer,

    causal_flag=False
)


In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# ---------------- CONFIG ----------------
candidate_dims = [6] * 3
D_graph = 4

inner_archive_size = 80
inner_offspring = 40
outer_archive_size = 40
outer_offspring = 40
inner_iters_per_outer = 50
outer_generations = 101
outer_cost_limit = 1000
inner_learning = 0.1
gamma_interlayer = 0.3
seed = 42
np.random.seed(seed)
class MetricsEvaluator:
    def __init__(self, data_matrix):
        self.data_matrix = data_matrix
        self.num_features = data_matrix.shape[1]
        self.W0, self.T0, self.U0 = 10.0, 100.0, 1.0
        self.P0 = 1.0  # baseline for patience

    def compute_node_metrics(self, node_idx, y=None):
        row = self.data_matrix[node_idx % self.data_matrix.shape[0]]
        k = self.num_features
        base = max(1, k // 3)
        wait_cols = slice(0, base)
        thr_cols = slice(base, 2*base)
        util_cols = slice(2*base, k)

        wait_signal = np.mean(row[wait_cols])
        throughput_signal = np.mean(row[thr_cols])
        util_signal = np.mean(row[util_cols])

        if y is None:
            y = np.array([0.5, 0.5, 0.5])
        else:
            y = np.array(y[:3]) if len(y) >= 3 else np.pad(y, (0, 3-len(y)), constant_values=0.5)

        # Original metrics
        wait = np.clip(self.W0*(1 + 1.2*wait_signal + 0.8*y[0]), 0, 100)
        throughput = np.clip(self.T0*(1 + 1.1*throughput_signal + 0.6*y[1] - 0.4*wait_signal), 0, 150)
        util = np.clip(self.U0 + 0.8*util_signal + 0.6*y[2], 0, 1)

        # --- New metric: patience ---
        patience_signal = np.mean(row)  # simple average as proxy
        patience = np.clip(self.P0 * (1 + 0.5*patience_signal + 0.5*y[0]), 0, 2)  # scaled like others

        # Combine into score
        score = -wait + throughput + util + patience  # add patience like others

        return {'wait': wait, 'throughput': throughput, 'util': util,
                'patience': patience, 'score': score}

# ---------------- INTER-LAYER MUTUAL INFORMATION ----------------
class InterLayer:
    def __init__(self, D_graph, max_inner_dim, inter_dim=None, edge_threshold=0.02, gamma=1.0, seed=42):
        np.random.seed(seed)
        self.D_graph = D_graph
        self.max_input = 2*max_inner_dim
        self.edge_threshold = edge_threshold
        self.gamma = gamma
        self.inter_dim = inter_dim if inter_dim is not None else max_inner_dim
        self.weights = {(i,j): np.random.uniform(-0.6,0.6,(self.inter_dim,self.max_input))
                        for i in range(D_graph) for j in range(D_graph) if i!=j}
        self.bias = {(i,j): np.random.uniform(-0.3,0.3,self.inter_dim)
                     for i in range(D_graph) for j in range(D_graph) if i!=j}

    def compute_edge_activation(self, i, j, nested_reps):
        concat = np.concatenate([nested_reps[i], nested_reps[j]])
        concat = np.pad(concat, (0, max(0,self.max_input-len(concat))))[:self.max_input]
        v = self.weights[(i,j)].dot(concat) + self.bias[(i,j)]
        return 1/(1+np.exp(-v))

    def build_activations(self, Gmat, nested_reps):
        acts = {}
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i==j: continue
                if abs(Gmat[i,j])>self.edge_threshold:
                    acts[(i,j)] = self.compute_edge_activation(i,j,nested_reps)
        return acts

    @staticmethod
    def pairwise_squared_corr(acts):
        if len(acts)<2: return 0.0
        A = np.stack(list(acts.values()))
        A_centered = A - A.mean(axis=1,keepdims=True)
        stds = np.sqrt(np.sum(A_centered**2,axis=1)/(A.shape[1]-1) + 1e-12)
        cov = A_centered @ A_centered.T / (A.shape[1]-1)
        corr = cov / (np.outer(stds,stds)+1e-12)
        np.fill_diagonal(corr,0)
        return float((corr**2).sum())

    def mi_for_graph(self, Gmat, nested_reps):
        acts = self.build_activations(Gmat,nested_reps)
        if not acts: return 0.0
        return self.gamma * self.pairwise_squared_corr(acts)

# ---------------- UNIFIED ACOR MULTIPLEX ----------------
class GDFCM:
    def __init__(self, candidate_dims, D_graph, inner_archive_size, inner_offspring,
                 outer_archive_size, outer_offspring, synthetic_targets, inner_learning,
                 gamma_interlayer=1.0, causal_flag=True):
        self.candidate_dims = candidate_dims
        self.D_graph = D_graph
        self.inner_archive_size = inner_archive_size
        self.inner_offspring = inner_offspring
        self.outer_archive_size = outer_archive_size
        self.outer_offspring = outer_offspring
        self.synthetic_targets = synthetic_targets
        self.inner_learning = inner_learning
        self.causal_flag = causal_flag

        self.nested_reps = [np.zeros(max(candidate_dims)) for _ in range(D_graph)]
        self.best_dim_per_node = [candidate_dims[0] for _ in range(D_graph)]
        self.inter_layer = InterLayer(D_graph, max_inner_dim=max(candidate_dims), gamma=gamma_interlayer)
        self.chosen_Gmat = np.random.uniform(-0.5,0.5,(D_graph,D_graph))
        np.fill_diagonal(self.chosen_Gmat,0)
        self.l2_before, self.l2_after = [], []

    # ---------- INNER LOOP (FCM) ----------
    def run_inner(self, node_idx, target, D_fcm,
              steps=100, lr_x=0.001, lr_y=0.001, lr_W=0.001):

        # two activation channels
        x = target.copy()
        y = target.copy()

        W = np.random.uniform(-0.6, 0.6, (D_fcm, D_fcm))
        np.fill_diagonal(W, 0)

        self.l2_before.append(np.linalg.norm(self.nested_reps[node_idx] - target))

        for _ in range(steps):

            z = W.dot(x)

            Theta_grad_z = 2*z - target

            # u = x, v = y
            Theta_grad_x = Theta_grad_z @ W + (y + 1)
            Theta_grad_y = (x + 1)

            Theta_grad_W = np.outer(Theta_grad_z, x)

            # updates
            x -= lr_x * np.clip(Theta_grad_x, -0.05, 0.05)
            y -= lr_y * np.clip(Theta_grad_y, -0.05, 0.05)

            x = np.clip(x, 0, 1)
            y = np.clip(y, 0, 1)

            W -= lr_W * np.clip(Theta_grad_W, -0.01, 0.01)
            np.fill_diagonal(W, 0)
            W = np.clip(W, -1, 1)

        self.nested_reps[node_idx] = x

        self.l2_after.append(np.linalg.norm(x - target))
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return x, y, W, mi_score

    # ---------- OUTER LOOP ----------
    def run_outer(self, outer_cost_limit=1000):
        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)
        node_metrics_list = []
        raw_scores = []

        # --- Compute node metrics per node ---
        for i, y in enumerate(self.nested_reps):
            metrics = metrics_evaluator.compute_node_metrics(i, y=y)
            node_metrics_list.append(metrics)
            raw_scores.append(metrics['score'])

        raw_scores = np.array(raw_scores)
        total_raw = raw_scores.sum()

        # --- Apply cap to raw metrics ---
        capped_total_raw = total_raw
        if total_raw > outer_cost_limit:
            scale_factor = outer_cost_limit / total_raw
            for metrics in node_metrics_list:
                for key in ['wait', 'throughput', 'util', 'patience', 'score']:
                    metrics[key] *= scale_factor
            raw_scores *= scale_factor
            capped_total_raw = outer_cost_limit

        # --- Compute Fuzzy Metric Tensor contribution ---
        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)
        D = self.D_graph

        # Only consider off-diagonal entries for inter-node interactions
        off_diag_mask = np.ones((D, D), dtype=bool)
        np.fill_diagonal(off_diag_mask, 0)
        fuzzy_score_offdiag = fuzzy_tensor[off_diag_mask].sum()

        # --- Compute per-node contribution ---
        node_contributions = np.zeros(D)
        for i in range(D):
            # Contribution from own metrics
            own_score = raw_scores[i]

            # Contribution from FMT interactions (row i -> others)
            fmt_contrib = fuzzy_tensor[i, :, :].sum() - fuzzy_tensor[i, i, :].sum()  # exclude self
            node_contributions[i] = own_score + self.inter_layer.gamma * fmt_contrib

        # --- Combine total score ---
        combined_score = node_contributions.sum()

        # --- Store for plotting / further analysis ---
        self.capped_node_metrics = node_metrics_list
        self.node_score_contributions = node_contributions

        return node_metrics_list, combined_score, node_contributions



    # ---------- FULL RUN ----------
    def run(self, outer_generations=outer_generations):
        final_metrics = None

        for gen in range(outer_generations):
            mi_scores = []

            # --- Inner loop per node ---
            for node_idx in range(self.D_graph):
                dim = self.best_dim_per_node[node_idx]
                target = self.synthetic_targets[node_idx][dim]
                _, _, _, mi_score = self.run_inner(node_idx, target, dim)
                mi_scores.append(mi_score)

            # --- Outer loop (now returns node contributions too) ---
            metrics_list, capped_score, node_contributions = self.run_outer()

            # --- Print node metrics ---
            print(f"\n--- Generation {gen} Metrics ---")
            for i, m in enumerate(metrics_list):
                print(
                    f"Node {i} | "
                    + " | ".join([f"{k}: {v:.2f}" for k, v in m.items()])
                )

            # --- Print node contributions ---
            print(f"\n--- Generation {gen} Node Contributions ---")
            for i, c in enumerate(node_contributions):
                print(f"Node {i}: Contribution = {c:.4f}")

            print(f"Outer Score (capped): {capped_score:.3f}")

            final_metrics = metrics_list

        return final_metrics


    # ---------- VISUALIZATIONS ----------
    def plot_pointwise_minmax_elite(self, top_k=21):
        plt.figure(figsize=(14,3))
        for i in range(self.D_graph):
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)
            y_min, y_max = reps.min(axis=0), reps.max(axis=0)
            y_sel = base
            y_true = self.synthetic_targets[i][self.best_dim_per_node[i]]
            if len(y_true)<len(y_sel):
                y_true = np.pad(y_true,(0,len(y_sel)-len(y_true)),"constant")
            else:
                y_true = y_true[:len(y_sel)]

            plt.subplot(1,self.D_graph,i+1)
            plt.fill_between(range(len(y_min)),y_min,y_max,color='skyblue',alpha=0.4,label='Elite Interval')
            plt.plot(y_sel,'k-',lw=2,label='Estimated')
            plt.plot(y_true,'r--',lw=2,label='True')
            plt.ylim(0,1.05)
            plt.title(f"Node {i+1}")
            if i==0: plt.legend()
        plt.tight_layout()
        plt.show()

    def plot_nested_activations(self):
        plt.figure(figsize=(12,3))
        for i,rep in enumerate(self.nested_reps):
            plt.subplot(1,self.D_graph,i+1)
            plt.bar(range(len(rep)),rep,color=plt.cm.plasma(rep))
            plt.ylim(0,1)
            plt.title(f"Node {i+1}")
        plt.tight_layout()
        plt.show()

    def plot_outer_fuzzy_graph(self):
        G = nx.DiGraph()
        for i in range(self.D_graph): G.add_node(i)
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i!=j and abs(self.chosen_Gmat[i,j])>0.02:
                    G.add_edge(i,j,weight=self.chosen_Gmat[i,j])
        node_sizes = [self.best_dim_per_node[i]*200 for i in range(self.D_graph)]
        edge_colors = ['green' if d['weight']>0 else 'red' for _,_,d in G.edges(data=True)]
        edge_widths = [abs(d['weight'])*3 for _,_,d in G.edges(data=True)]
        pos = nx.circular_layout(G)
        plt.figure(figsize=(6,6))
        nx.draw(G,pos,node_size=node_sizes,node_color='skyblue',
                edge_color=edge_colors,width=edge_widths,arrows=True,with_labels=True)
        plt.title("Outer Fuzzy Multiplex Graph")
        plt.show()
# ---------------- INTERACTIONS INSPECTOR ----------------

    def print_interactions(self, return_tensor=True, verbose=True):
            D_graph = self.D_graph
            inter_dim = self.inter_layer.inter_dim
            inter_tensor = np.zeros((D_graph, D_graph, inter_dim))

            acts = self.inter_layer.build_activations(self.chosen_Gmat, self.nested_reps)
            if not acts:
                if verbose:
                    print("No active edges above threshold.")
                return inter_tensor if return_tensor else None

            for (i, j), vec in acts.items():
                inter_tensor[i, j, :] = vec
                if verbose:
                    act_str = ", ".join([f"{v:.3f}" for v in vec])
                    print(f"Node {i} -> Node {j}: [{act_str}]")
            return inter_tensor if return_tensor else None

        # Move these outside of print_interactions (class-level)
    def print_l2_summary(self):
            print("\nL2 Distances to Target per Node:")
            for idx, (before, after) in enumerate(zip(self.l2_before, self.l2_after)):
                print(f"Node {idx}: Before={before:.4f}, After={after:.4f}")

    def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
            metrics_keys = ['wait', 'throughput', 'util', 'patience']
            D = self.D_graph
            num_metrics = len(metrics_keys)
            tensor = np.zeros((D, D, num_metrics))

            metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

            node_metrics = []
            for i, rep in enumerate(self.nested_reps):
                metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
                node_metrics.append(np.array([metrics[k] for k in metrics_keys]))
            node_metrics = np.array(node_metrics)  # (D, num_metrics)

            for i in range(D):
                for j in range(D):
                    if i == j:
                        tensor[i, j, :] = node_metrics[j]
                    else:
                        weight = np.clip(abs(self.chosen_Gmat[i, j]), 0, 1)
                        tensor[i, j, :] = weight * node_metrics[j]

            if normalize:
                tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

            if verbose:
                print("Fuzzy Metric Tensor shape:", tensor.shape)

            return tensor

    def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
            """
            Computes a Fuzzy Metric Tensor (D_graph x D_graph x num_metrics)
            using current nested reps and node metrics.
            Each slice [i,j,:] represents metrics of node j (optionally weighted by Gmat[i,j])
            """
            metrics_keys = ['wait', 'throughput', 'util', 'patience']
            D = self.D_graph
            num_metrics = len(metrics_keys)
            tensor = np.zeros((D, D, num_metrics))

            metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

            node_metrics = []
            for i, rep in enumerate(self.nested_reps):
                metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
                node_metrics.append(np.array([metrics[k] for k in metrics_keys]))
            node_metrics = np.array(node_metrics)  # (D, num_metrics)

            for i in range(D):
                for j in range(D):
                    if i==j:
                        tensor[i,j,:] = node_metrics[j]
                    else:
                        weight = np.clip(abs(self.chosen_Gmat[i,j]), 0, 1)
                        tensor[i,j,:] = weight * node_metrics[j]

            if normalize:
                tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

            if verbose:
                print("Fuzzy Metric Tensor shape:", tensor.shape)

            return tensor
    def plot_fuzzy_metric_tensor_heatmaps(self, fuzzy_tensor=None, metrics_keys=['wait','throughput','util','patience']):
        """
        Plot a heatmap panel for each metric in the FMT.
        Rows: source node i
        Columns: target node j
        """
        if fuzzy_tensor is None:
            fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

        D = self.D_graph
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4*num_metrics,4))
        if num_metrics == 1: axes = [axes]

        for k, key in enumerate(metrics_keys):
            data = fuzzy_tensor[:,:,k]
            im = axes[k].imshow(data, cmap='viridis', vmin=0, vmax=1)
            for i in range(D):
                for j in range(D):
                    axes[k].text(j,i,f"{data[i,j]:.2f}",ha='center',va='center',color='white',fontsize=9)
            axes[k].set_xticks(range(D))
            axes[k].set_yticks(range(D))
            axes[k].set_xticklabels([f'Node {j}' for j in range(D)])
            axes[k].set_yticklabels([f'Node {i}' for i in range(D)])
            axes[k].set_title(f'FMT - {key}')

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Normalized Metric Value')
        plt.tight_layout()
        plt.show()

    def compute_fmt_with_elite_bounds(self, top_k=21):
        """
        Computes FMT bounds using pointwise min/max across elite solutions.
        Returns tensor shape (D,D,num_metrics,2) [lower, upper].
        """
        metrics_keys = ['wait','throughput','util','patience']
        D = self.D_graph
        num_metrics = len(metrics_keys)
        tensor_bounds = np.zeros((D,D,num_metrics,2))

        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

        for i in range(D):
            # Generate top_k perturbations around current nested_rep (like in plot_pointwise_minmax_elite)
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)

            # Compute node metrics for each perturbed solution
            metrics_matrix = np.zeros((top_k, num_metrics))
            for idx, rep in enumerate(reps):
                m = metrics_evaluator.compute_node_metrics(i, y=rep)
                metrics_matrix[idx,:] = [m[k] for k in metrics_keys]

            # Compute pointwise min/max across elite solutions
            lower_i = metrics_matrix.min(axis=0)
            upper_i = metrics_matrix.max(axis=0)

            # Fill bounds tensor for all source nodes (i->j)
            for j in range(D):
                tensor_bounds[i,j,:,0] = lower_i
                tensor_bounds[i,j,:,1] = upper_i

        return tensor_bounds


    def plot_fmt_with_bounds(self, fmt_tensor_bounds):
        D = self.D_graph
        metrics_keys = ['wait','throughput','util','patience']
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4*num_metrics,4))
        if num_metrics == 1: axes = [axes]

        for k, key in enumerate(metrics_keys):
            lower = fmt_tensor_bounds[:,:,k,0]
            upper = fmt_tensor_bounds[:,:,k,1]
            mean_vals = (lower+upper)/2
            range_vals = upper-lower
            max_range = range_vals.max() if range_vals.max()>0 else 1.0
            alphas = 0.2 + 0.8 * range_vals/max_range

            im = axes[k].imshow(mean_vals, cmap='viridis', vmin=0, vmax=mean_vals.max())
            for i in range(D):
                for j in range(D):
                    alpha_val = np.clip(1-alphas[i,j],0,1)
                    rect = plt.Rectangle((j-0.5,i-0.5),1,1,color='white',alpha=alpha_val)
                    axes[k].add_patch(rect)
                    axes[k].text(j,i,f"{lower[i,j]:.1f}\n{upper[i,j]:.1f}",ha='center',va='center',fontsize=8)
            axes[k].set_title(f'FMT Bounds - {key}')
            axes[k].set_xticks(range(D))
            axes[k].set_yticks(range(D))
            axes[k].set_xticklabels([f'Node {j}' for j in range(D)])
            axes[k].set_yticklabels([f'Node {i}' for i in range(D)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Mean Metric Value')
        plt.tight_layout()
        plt.show()
    def plot_node_score_contribution(self, metrics_keys=['wait','throughput','util','patience']):
        """
        Plot per-node total score contribution in the SAME STYLE as the FMT plots:
            - uses imshow
            - one panel for: raw, FMT, and total stacked
            - diagonal masked
            - annotated cells
            - node contribution highlighted like your FMT code
        """
        # ---------------------------------------------------------------------
        # 1. Collect node contributions from run_outer()
        # ---------------------------------------------------------------------
        _, _, node_contributions = self.run_outer()
        node_contributions = np.array(node_contributions)
        D = len(node_contributions)

        # ---------------------------------------------------------------------
        # 2. Recompute FMT influence (same style as your FMT plots)
        # ---------------------------------------------------------------------
        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)
        total_tensor = fuzzy_tensor.sum(axis=2)           # sum over metrics
        fmt_tensor = total_tensor.copy()
        np.fill_diagonal(fmt_tensor, 0)                   # mask diagonal

        fmt_per_node = fmt_tensor.sum(axis=1)             # row sum
        raw_per_node = node_contributions - fmt_per_node  # everything else

        # Construct matrices for plotting (D×D)
        raw_matrix = np.zeros((D, D))
        fmt_matrix = fmt_tensor
        total_matrix = raw_matrix + fmt_matrix            # raw only on diagonal? no → distribute raw as row diag
        np.fill_diagonal(raw_matrix, raw_per_node)
        total_matrix = raw_matrix + fmt_matrix

        # ---------------------------------------------------------------------
        # 3. Plot - 3 subplots in SAME STYLE as FMT panels
        # ---------------------------------------------------------------------
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))

        matrices = [raw_matrix, fmt_matrix, total_matrix]
        titles = ["Raw Node Contribution", "FMT Interaction Contribution", "Total Contribution"]

        for ax, mat, title in zip(axes, matrices, titles):

            im = ax.imshow(mat, cmap='viridis', vmin=np.min(mat), vmax=np.max(mat))

            # annotate values
            for i in range(D):
                for j in range(D):
                    val = mat[i, j]
                    ax.text(j, i, f"{val:.2f}", ha='center',
                            va='center', color='white', fontsize=8)

            ax.set_title(title)
            ax.set_xticks(range(D))
            ax.set_xticklabels([f"Node {i+1}" for i in range(D)])
            ax.set_yticks(range(D))
            ax.set_yticklabels([f"Node {i+1}" for i in range(D)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04,
                    label='Contribution Value')

        plt.tight_layout()
        plt.show()



# ---------------- USAGE ----------------
if __name__ == "__main__":
    optimizer = GDFCM(
        candidate_dims, D_graph,
        inner_archive_size, inner_offspring,
        outer_archive_size, outer_offspring,
        synthetic_targets,
        inner_learning, gamma_interlayer=gamma_interlayer,
        causal_flag=False
    )
    metrics_list = optimizer.run()
    optimizer.plot_pointwise_minmax_elite()
    optimizer.plot_nested_activations()
    # Compute FMT with elite bounds
    fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=21)

# Plot as heatmaps
    optimizer.plot_fmt_with_bounds(fmt_elite_bounds)

    # Compute fuzzy multiplex tensor
    fmt_tensor = optimizer.compute_fuzzy_metric_tensor(normalize=True)
    optimizer.plot_fuzzy_metric_tensor_heatmaps(fmt_tensor)

    # Compute FMT with bounds (minimax elite intervals)
    optimizer.plot_node_score_contribution()
    optimizer.plot_outer_fuzzy_graph()
  #  optimizer.print_interactions()
    tensor = optimizer.print_interactions()

    print("Tensor shape:", tensor.shape,'\n',tensor)
    import networkx as nx
    import numpy as np
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
D_graph = len(optimizer.nested_reps)
tensor = optimizer.print_interactions(return_tensor=True, verbose=False)

# ---------------- Outer nodes (hubs) ----------------
G_outer = nx.DiGraph()
for i in range(D_graph):
    G_outer.add_node(i)
for i in range(D_graph):
    for j in range(D_graph):
        if i != j and np.any(tensor[i,j,:] != 0):
            # Shift to signed weights: 0.5 -> 0, <0.5 negative, >0.5 positive
            mean_weight = 2 * (np.mean(tensor[i,j,:]) - 0.5)
            G_outer.add_edge(i, j, weight=mean_weight)

# Outer spring layout
pos_outer_2d = nx.circular_layout(G_outer, scale=5)
pos_outer = np.array([[x, y, 0] for x, y in pos_outer_2d.values()])

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')

# Plot outer nodes
for i in range(D_graph):
    ax.scatter(*pos_outer[i], s=300, color='skyblue')
    ax.text(*pos_outer[i], f'Node {i}', color='black')

# Plot outer edges with positive/negative colors

for i, j, data in G_outer.edges(data=True):
    x_vals = [pos_outer[i,0], pos_outer[j,0]]
    y_vals = [pos_outer[i,1], pos_outer[j,1]]
    z_vals = [pos_outer[i,2], pos_outer[j,2]]

    # Positive = bright green, Negative = bright red
    color = 'green' if data['weight'] > 0 else 'red'
    linewidth = 2 + 4*abs(data['weight'])  # scale width by magnitude
    ax.plot(x_vals, y_vals, z_vals, color=color, linewidth=linewidth)
# ---------------- Inner FCMs (small circular around hub) ----------------
for i, rep in enumerate(optimizer.nested_reps):
    dims = len(rep)
    angle = np.linspace(0, 2*np.pi, dims, endpoint=False)
    radius = 0.8  # small circle
    xs = pos_outer[i,0] + radius * np.cos(angle)
    ys = pos_outer[i,1] + radius * np.sin(angle)
    zs = pos_outer[i,2] + rep  # activation as height

    # Plot inner nodes
    ax.scatter(xs, ys, zs, c=rep, cmap='plasma', s=50)

    # Connect inner nodes in circle
    for k in range(dims):
        ax.plot([xs[k], xs[(k+1)%dims]], [ys[k], ys[(k+1)%dims]], [zs[k], zs[(k+1)%dims]], color='gray', alpha=0.5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Activation')
ax.set_title('Outer Nodes with Inner FCMs (Signed correlations)')
plt.show()


